# Making Predictions Using the Naive Bayes Algorithm
*Curtis Miller*

The **naive Bayes** algorithm is based on Bayes' theorem. Let's suppose that our data uses features $X_1, X_2, ..., X_K$ and $Y$ is the target variable. One form to describe the naive Bayes classifier is:

$$P(Y = y | X_1 = x_1, ..., X_K = x_K) = \frac{P(X_1 = x_1, ..., X_K = x_K | Y = y) P(Y = y)}{P(X_1 = x_1, ..., X_K = x_K)}$$

The "naive" part of the naive Bayes classifier is to make the (unrealistic) assumption that all the features are **independent** random variables; that is, information about one feature provides essentially no information about the other, and $P(X_i = x_i | X_j = x_j) = P(X_i = x_i)$ when $i \neq j$ (but we *don't* assume the features are independent of the target variable). Under this assumption, $P(X_i = x_i, X_j = x_j) = P(X_i = x_i)P(X_j = x_j)$ and we can rewrite Bayes theorem as

$$P(Y = y | X_1 = x_1, ..., X_K = x_K) = \frac{P(Y = y) \prod_{k = 1}^{K} P(X_k = x_k | Y = y)}{\prod_{k = 1}{K} P(X_k = x_k)} \propto P(Y = y) \prod_{k = 1}^{K} P(X_k = x_k | Y = y)$$

The quantities in the above expression (behind the $\propto$ symbol) are easily estimated from a dataset; this is what is done when training the algorithm.

When predicting, we observe features $x_1, ..., x_K$ for a data point. The predicted label $y$ is the label that maximizes $P(Y = y) \prod_{k = 1}^{K} P(X_k = x_k | Y = y)$; that is, if $\hat{y}$ represents our prediction, then

$$\hat{y} = \arg \max_y P(Y = y) \prod_{k = 1}^{K} P(X_k = x_k | Y = y)$$

Notice that what I have written works only when $X_k$ are all discrete; this algorithm will work for categorical features (like passenger class or sex) or variables that take countable values (like number of children aboard), but it won't work for features we may consider continuous (like fare paid) since in that case $P(X_k = x_k | Y = y)$ always. We fix this problem by replacing, for continuous features, $P(X_k = x_k | Y = y)$ with $f_k(x_k | y)$, where $f( \cdot )$ is a probability density function. A common choice is the Gaussian density:

$$f_k(x_k | y) = \frac{1}{\sqrt{2 \pi \sigma^2_{k,y}}} \exp\left(-\frac{(x_k - \mu_{k,y})^2}{2 \sigma_{k,y}^2}\right)$$

(Note that $\exp(x) = e^x$.) The parameters $\mu_{k,y}$ and $\sigma_{k,y}$ are estimated from the data.

If $U_1, ..., U_I$ are discrete variables and $V_1, ..., V_J$ continuous, we can rewrite the naive Bayes classifier like so:

$$\hat{y} = \arg \max_y P(Y = y) \prod_{i = 1}^{I} P(U_i = u_i | Y = y) \prod_{j = 1}^{J} f_j(v_j | y)$$

Hyperparameters for the naive Bayes algorithm are the prior distributions for all features and probabilities mentioned here, before observing data. In particular we may want to choose a prior probability for $P(Y = y)$.

In this notebook I will be implementing Bernoulli naive Bayes, where we don't allow for any continuous random variables; every variable takes one of two values. This means we will need to implement binning for continuous variables and break up count variables into each observed count. This requires preprocessing the data. The choice of bins also acts like a hyperparameter.

Bernoulli naive Bayes is implemented in **scikit-learn** through the `BernoulliNB` object.

## Linear Separability

KNN, decision trees, and decision forests don't assume much about the underlying data, but from this point on the classifiers I consider (including the naive Bayes algorithm) assume that the data is **linearly separable**. That is, data with different labels can be separated by a straight line in 2D space, or a hyperplane in N-dimensional space (a hyperplane is a general notion of a line). Data is usually considered to be lying in a space that has more than even three dimensions If this assumption is violated, the algorithm may struggle, if not fail outright, to correctly learn and classify the data.

In general it's difficult to check whether data is linearly separable. Visualization may be useful for determining this.

## Preprocessing the Data

We will continue our work with the *Titanic* dataset.

In [ ]:
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report

In [ ]:
titanic = pd.read_csv("titanic.csv")
titanic.head()

Here we would model `Pclass`, `Sex`, `Siblings/Spouses Aboard`, and `Parents/Children Aboard` as discrete variable, while `Age` and `Fare` should be considered continuous. We will need to bin `Age` and `Fare` in order to be able to use `BernoulliNB`. Our work with decision trees may suggest what bins to use (we may also want to use cross-validation).

In [ ]:
pd.cut(titanic.Age, bins=[-1, 2, titanic.Age.max() + 1]).head()

In [ ]:
pd.cut(titanic.Fare, bins=[0, 23.35, titanic.Fare.max() + 1]).head()

In [ ]:
titanic = titanic.assign(Age_cat=(titanic.Age <= 2), Fare_cat=(titanic.Fare <= 23.35))
titanic.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)
titanic.drop(['Age', 'Fare', 'Name'], axis=1, inplace=True)
titanic.head()

In [ ]:
titanic_train, titanic_test = train_test_split(titanic)
titanic_train.head()

## Training a Naive Bayes Algorithm

Let's now fit a Bernoulli naive Bayes algorithm.

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
bnb = BernoulliNB(alpha=0,    # Additive smoothing parameter; setting to 0 for no smoothing
                  fit_prior=False,     # Don't learn a prior distribution for the label
                  class_prior=None)    # Don't have prior distributions for features
bnb = bnb.fit(titanic_train.drop("Survived", axis=1), titanic_train.Survived)
print(classification_report(titanic_train.Survived, bnb.predict(titanic_train.drop("Survived", axis=1))))

Now let's see the algorithm's predictive accuracy on the test set.

In [ ]:
survived_test_predict = bnb.predict(titanic_test.drop("Survived", axis=1))
print(classification_report(titanic_test.Survived, survived_test_predict))

In-sample and out-of-sample performance for this dataset are similar. That demonstrates one feature of linear models that data scientists like: linear models tend to generalize well.

That said, let's see if we can get better performance.